### Tarea 2

In [1]:
import numpy as np
import zipfile
import tarfile
import os
from collections import defaultdict
import nltk
from nltk.util import ngrams
from tqdm import tqdm 
import xml.etree.ElementTree as ET
import re
from nltk.tokenize import sent_tokenize, word_tokenize
import random
from collections import Counter

# Download NLTK data for tokenization
nltk.download('punkt')

c:\Users\USUARIO\anaconda3\envs\anteia\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\USUARIO\anaconda3\envs\anteia\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\USUARIO\anaconda3\envs\anteia\lib\site-packages\numpy\.libs\libopenblas64__v0.3.23-246-g3d31191b-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Se debe descaragr el archivo "Datasets.zip" en el mismo directorio de el notebook

## Extracción Dataset

In [2]:
compressed_file = "Datasets.zip"
with zipfile.ZipFile(compressed_file, 'r') as zip_ref:
    folder_name = os.path.splitext(compressed_file)[0]  # Remove the ".zip" extension
    target_folder = os.path.join(folder_name)
    
    if not os.path.exists(target_folder):
        # Create the folder within the target directory
        os.mkdir(target_folder)

    
        # Extract all files to the target folder
        zip_ref.extractall(target_folder)

print("Extracción completada")

Extracción completada


In [3]:
compressed_file = "Datasets\\20news-18828.tar.gz"
folder_name = os.path.splitext(os.path.splitext(compressed_file)[0])[0]  # Remove the ".tar.gz" extension

if not os.path.exists(folder_name):
    os.mkdir(folder_name)

# Extract all files from the TAR.GZ archive to the target folder without creating an additional subfolder
with tarfile.open(compressed_file, 'r:gz') as tar_ref:
    members = tar_ref.getmembers()
    tar_ref.extractall(path=folder_name, members=members)

print("Extracción completada")

Extraction completed


In [4]:
compressed_file = "Datasets\\BAC\\blogs.zip"
with zipfile.ZipFile(compressed_file, 'r') as zip_ref:
    folder_name = os.path.splitext(compressed_file)[0]  # Remove the ".zip" extension
    target_folder = os.path.join(folder_name)
    
    if not os.path.exists(target_folder):
        # Create the folder within the target directory
        os.mkdir(target_folder)

    
        # Extract all files to the target folder
        zip_ref.extractall(target_folder)

print("Extracción completada")

Extracción completada


## Creación de Archivos consolidados

In [5]:
# Define the paths to the dataset folders
path_20N = "Datasets\\20news-18828"

# Define the paths to the consolidated files
consolidated_20N_file = "consolidated_20N.txt"

# Function to read and consolidate documents in a folder
def consolidate_documents(folder_path, output_file):
    with open(output_file, "w", encoding="utf-8") as output:
        for root, _, files in os.walk(folder_path):
            for file_name in tqdm(files, desc="Processing Documents"):  # Add tqdm here
                file_path = os.path.join(root, file_name)
                with open(file_path, "r", encoding="utf-8", errors="ignore") as file:
                    content = file.read()
                    output.write(content)
                    output.write("\n")  # Add a newline to separate documents

# Consolidate documents from the "20news-18828" dataset with a progress bar
consolidate_documents(path_20N, consolidated_20N_file)
print("Consolidated 20N dataset.")

Processing Documents: 0it [00:00, ?it/s]
Processing Documents: 0it [00:00, ?it/s]
Processing Documents: 100%|██████████| 628/628 [00:02<00:00, 234.12it/s]

Consolidated 20N dataset.


In [6]:
# Define the path to the BAC dataset folder
path_BAC = "Datasets\\BAC"

# Define the path to the consolidated text file
consolidated_BAC_file = "consolidated_BAC.txt"

# Function to consolidate text from XML documents in a folder
def consolidate_text_from_xml(folder_path, output_file):
    with open(output_file, "w", encoding="utf-8") as output:
        for root, _, files in os.walk(folder_path):
            for file_name in tqdm(files, desc="Processing XML files"):  # Add tqdm here
                file_path = os.path.join(root, file_name)
                # Check if the file is within the "blogs" folder and has a ".xml" extension
                if "blogs" in root and file_name.endswith(".xml"):
                    try:
                        tree = ET.parse(file_path)
                        root_element = tree.getroot()
                        for post_element in root_element.findall(".//post"):
                            content = post_element.text.strip() if post_element.text else ""
                            # Write the extracted content to the output file
                            output.write(content)
                            output.write("\n")  # Add a newline to separate documents
                    except ET.ParseError as e:
                        with open(file_path, "r", encoding="utf-8", errors="ignore") as file_:
                            xml_content = file_.read()

                        # Define a regular expression pattern to match content between <date> and </date> tags
                        pattern = r'<date>.*?</date>'

                        # Use re.sub() to replace matched patterns with an empty string
                        xml_content_filtered = re.sub(pattern, '', xml_content, flags=re.DOTALL)

                        xml_content_filtered = re.sub(r'<.*?>', '', xml_content_filtered)
                        # Remove multiple spaces with a single space
                        xml_content_filtered = re.sub(r'\s+', ' ', xml_content_filtered)

                        xml_content_filtered = xml_content_filtered.replace('\n', '')
                        output.write(xml_content_filtered.strip())
                        output.write("\n")  # Add a newline to separate documents
                        # print(f"Error parsing {file_path}: {e}, using raw file...")

# Path to the "blogs" folder within the BAC dataset
blogs_path_BAC = os.path.join(path_BAC, "blogs")

# Consolidate text content from XML documents within the "blogs" folder of the BAC dataset
consolidate_text_from_xml(blogs_path_BAC, consolidated_BAC_file)
print("Consolidated BAC dataset (text content from XML documents within 'blogs' folder).")

Processing XML files: 0it [00:00, ?it/s]
Processing XML files: 100%|██████████| 19320/19320 [02:07<00:00, 151.94it/s]

Consolidated BAC dataset (text content from XML documents within 'blogs' folder).


## Normalización

In [16]:
# Define a function to preprocess and tokenize text with a stopping parameter
def preprocess_and_tokenize(text, max_chars=None):
    # Truncate the input text if it exceeds the specified maximum characters
    if max_chars is not None and len(text) > max_chars:
        text = text[:max_chars]

    # Tokenize text by sentence
    sentences = sent_tokenize(text)

    # Token frequency counter
    token_freq = Counter()

    # Define a list to store the preprocessed sentences
    preprocessed_sentences = []

    # Initialize a sentence count
    sentence_count = 0

    for sentence in sentences:
        # Add sentence start and end tags
        sentence = "<s> " + sentence + " </s>"

        # Tokenize words within the sentence
        words = word_tokenize(sentence)

        # Normalize and process each word
        processed_words = []
        for word in words:
            # Normalize word (convert to lowercase)
            word = word.lower().strip()

            # Check if the word is a number and replace with "NUM"
            if re.match(r'^[0-9]+$', word):
                word = "NUM"

            # Update token frequency
            token_freq[word] += 1

            processed_words.append(word)

        # Join the processed words back into a sentence
        preprocessed_sentence = " ".join(processed_words)

        # Remove extra spaces between "<", "s", and ">"
        preprocessed_sentence = preprocessed_sentence.replace("< ", "<").replace(" >", ">")

        # Add the preprocessed sentence to the list
        preprocessed_sentences.append(preprocessed_sentence)

        # Increment the sentence count
        sentence_count += 1

    # Replace tokens with unit frequency as "<UNK>"
    for i in range(len(preprocessed_sentences)):
        words = preprocessed_sentences[i].split()
        for j in range(len(words)):
            if token_freq[words[j]] == 1 and words[j] != "NUM":
                words[j] = "<UNK>"
        preprocessed_sentences[i] = " ".join(words)

    # Return the preprocessed sentences
    return preprocessed_sentences

text = "This is an example sentence. It contains numbers like 123 and 456789. Some words may be removed. Some words may be removed x 2"

# Preprocess and tokenize the text
preprocessed_sentences = preprocess_and_tokenize(text, max_chars=100)

# Print the preprocessed sentences
for idx, sentence in enumerate(preprocessed_sentences):
    print("{}: {}".format(idx+1, sentence))


1: <s> <UNK> <UNK> <UNK> <UNK> <UNK> . </s>
2: <s> <UNK> <UNK> <UNK> <UNK> NUM <UNK> NUM . </s>
3: <s> <UNK> <UNK> <UNK> <UNK> <UNK> . </s>
4: <s> <UNK> </s>


## Partición del dataset

In [22]:
#maximo numero de caracteres o nunca termina 100.000.000 max
max_chars = 100000000
# Load the consolidated files
with open("consolidated_20N.txt", "r", encoding="utf-8") as file:
    sentences_20N = file.read()
preprocessed_sentences_20N = preprocess_and_tokenize(sentences_20N, max_chars=max_chars)

with open("consolidated_BAC.txt", "r", encoding="utf-8") as file:
    sentences_BAC = file.read()
preprocessed_sentences_BAC = preprocess_and_tokenize(sentences_BAC, max_chars=max_chars)

# Function to split sentences into training and testing sets
def split_sentences(sentences, train_ratio):
    random.shuffle(sentences)
    split_index = int(len(sentences) * train_ratio)
    train_sentences = sentences[:split_index]
    test_sentences = sentences[split_index:]
    return train_sentences, test_sentences

# Define the training ratio (80%)
train_ratio = 0.8

# Split sentences for 20N dataset
train_sentences_20N, test_sentences_20N = split_sentences(preprocessed_sentences_20N, train_ratio)

# Split sentences for BAC dataset
train_sentences_BAC, test_sentences_BAC = split_sentences(preprocessed_sentences_BAC, train_ratio)

# Define a group code
group_code = "group_code" 

# Create training and testing files
with open(f"20N_{group_code}_training.txt", "w", encoding="utf-8") as file:
    file.writelines(train_sentences_20N)

with open(f"20N_{group_code}_testing.txt", "w", encoding="utf-8") as file:
    file.writelines(test_sentences_20N)

with open(f"BAC_{group_code}_training.txt", "w", encoding="utf-8") as file:
    file.writelines(train_sentences_BAC)

with open(f"BAC_{group_code}_testing.txt", "w", encoding="utf-8") as file:
    file.writelines(test_sentences_BAC)

# Print confirmation messages
print("Training and testing files created for 20N and BAC datasets.")

Training and testing files created for 20N and BAC datasets.


In [23]:
# Delete variables to free up memory
del sentences_20N
del preprocessed_sentences_20N
del sentences_BAC
del preprocessed_sentences_BAC
del train_sentences_20N
del test_sentences_20N
del train_sentences_BAC
del test_sentences_BAC

## Modelos N-gramas

In [24]:
# Specify the folder name
n_grams_folder = "n_grams"

# Check if the folder already exists
if not os.path.exists(n_grams_folder):
    # If it doesn't exist, create it
    os.mkdir(n_grams_folder)
    print(f"Folder '{n_grams_folder}' created successfully.")
else:
    print(f"Folder '{n_grams_folder}' already exists.")

Folder 'n_grams' already exists.


In [25]:
# Define a function to calculate Laplace-smoothed probabilities for n-grams
def laplace_smoothing(ngram_counts, total_tokens, vocab_size, alpha=1.0):
    smoothed_probabilities = {}
    for ngram, count in ngram_counts.items():
        smoothed_prob = (count + alpha) / (total_tokens + alpha * vocab_size)
        smoothed_probabilities[ngram] = smoothed_prob

    return smoothed_probabilities

# Define a function to generate n-grams from a list of tokens
def generate_ngrams(tokens, n):
    ngrams = []
    for i in range(len(tokens) - n + 1):
        ngram = tuple(tokens[i:i + n])
        ngrams.append(ngram)
    return ngrams

# Define a function to build and save N-gram models with Laplace smoothing
def build_and_save_ngram_model(sentences, n, output_file):
    # Tokenize the sentences into words
    tokens = sentences.split(" ")

    # Generate n-grams
    ngrams = generate_ngrams(tokens, n)

    # Calculate n-gram counts
    ngram_counts = Counter(ngrams)

    # Calculate vocabulary size
    vocab_size = len(set(tokens))

    # Calculate total number of tokens
    total_tokens = len(tokens)

    # Apply Laplace smoothing to calculate smoothed probabilities
    smoothed_probabilities = laplace_smoothing(ngram_counts, total_tokens, vocab_size)

    # Save the smoothed probabilities to the output file
    with open(output_file, "w", encoding="utf-8") as file:
        for ngram, prob in smoothed_probabilities.items():
            ngram_str = " ".join(ngram)
            file.write(f"{ngram_str}\t{prob}\n")

In [26]:
# Paths to preprocessed training files
preprocessed_training_file_20N = f"20N_{group_code}_training.txt"
preprocessed_training_file_BAC = f"BAC_{group_code}_training.txt"

# Load preprocessed training data from the files
with open(preprocessed_training_file_20N, "r", encoding="utf-8") as file:
    train_sentences_20N = file.read()

with open(preprocessed_training_file_BAC, "r", encoding="utf-8") as file:
    train_sentences_BAC = file.read()

# Paths to training files
training_file_20N = f"20N_{group_code}_training.txt"
training_file_BAC = f"BAC_{group_code}_training.txt"

# Paths to output files
output_file_20N_unigrams = f"20N_{group_code}_unigrams.txt"
output_file_20N_bigrams = f"20N_{group_code}_bigrams.txt"
output_file_20N_trigrams = f"20N_{group_code}_trigrams.txt"
output_file_BAC_unigrams = f"BAC_{group_code}_unigrams.txt"
output_file_BAC_bigrams = f"BAC_{group_code}_bigrams.txt"
output_file_BAC_trigrams = f"BAC_{group_code}_trigrams.txt"

# Build and save N-gram models with Laplace smoothing
build_and_save_ngram_model(train_sentences_20N, 1, os.path.join(n_grams_folder,output_file_20N_unigrams))
print("1-gram model 20N")
build_and_save_ngram_model(train_sentences_20N, 2, os.path.join(n_grams_folder,output_file_20N_bigrams))
print("2-gram model 20N")
build_and_save_ngram_model(train_sentences_20N, 3, os.path.join(n_grams_folder,output_file_20N_trigrams))
print("3-gram model 20N")
build_and_save_ngram_model(train_sentences_BAC, 1, os.path.join(n_grams_folder,output_file_BAC_unigrams))
print("1-gram model BAC")
build_and_save_ngram_model(train_sentences_BAC, 2, os.path.join(n_grams_folder,output_file_BAC_bigrams))
print("2-gram model BAC")
build_and_save_ngram_model(train_sentences_BAC, 3, os.path.join(n_grams_folder,output_file_BAC_trigrams))
print("3-gram model BAC")

# Print confirmation message
print("N-gram models with Laplace smoothing generated and saved.")

1-gram model 20N
2-gram model 20N
3-gram model 20N
1-gram model BAC
2-gram model BAC
3-gram model BAC
N-gram models with Laplace smoothing generated and saved.
